Attribute lookup chain rewiev

In [1]:
class Child:
    name='Andrew'
    def __init__(self, name):
        self.name = name

In [2]:
Child.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Child.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Child' objects>,
              '__weakref__': <attribute '__weakref__' of 'Child' objects>,
              '__doc__': None})

In [3]:
c=Child('Anthony')

In [4]:
c.__dict__

{'name': 'Anthony'}

In [5]:
c.name # so first of all, python will try to find an attr in class and if not found - uses class var

'Anthony'

In [11]:
class Child:
    name='Andrew'
    def __init__(self, name=None):
        if name:
            self.name = name
v=Child('Vladimir')

In [12]:
v.name

'Vladimir'

In [13]:
v=Child()

In [14]:
v.name

'Andrew'

In [25]:
class GrandParent(object):
    name='Robert'
    
    
class Parent(GrandParent):
    pass
    #name='James'
class Child(Parent):
    # name='Andrew'
    def __init__(self, name=None):
        if name:
            self.name = name
v=Child()

In [27]:
v.name # it will lookup in parent class if not found in child

'Robert'

In [ ]:
# __getattr__ implemented in object

The descriptor protocol

In [ ]:
class Descriptor:
    pass

In [ ]:
# protocol -> contract between our objects and python

In [ ]:
# the descriptor protocol:
# __get__()
# __set__()
# __delete__()

In [ ]:
class Descriptor:
    def __get__(self, instance, owner):
        pass
    def __set__(self, instance, value):
        pass
    def __delete__(self, instance):
        pass
    # so the descriptor is the object that implements that methods

Using a descriptor

In [ ]:
# ORM -> OO vs SQL

In [ ]:
# we want to be able to define a persontable class that has a first name attr that is text of max len 200

In [34]:
class TextField:
    def __init__(self,length):
        self.length=length
    def __get__(self,instance,owner):
        return self.value
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        self.value=value
        
    def __delete__(self,instance):
        pass

In [35]:
class PersonTable:
    first_name=TextField(200)

In [36]:
p=PersonTable()

In [37]:
p.first_name='aaaaaa'

In [38]:
p.first_name

'aaaaaa'

In [ ]:
# binding behaviour

In [39]:
class PersonTable:
    first_name=TextField(200)
    def __init__(self, first_name):
        self.__dict__['first_name'] = first_name

In [41]:
p=PersonTable('Robbie')
p.first_name='Liam'

In [42]:
p.first_name

'Liam'

In [43]:
p.__dict__

{'first_name': 'Robbie'}

Descriptor storage

In [54]:
class TextField:
    def __init__(self,length):
        self.length=length
        self._data={}
    def __get__(self,instance,owner):
        return self._data.get(instance,None)
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        #self.value=value
        self._data[instance]=value # memory leak
        
    def __delete__(self,instance):
        pass
class PersonTable:
    first_name=TextField(200)

In [55]:
p1=PersonTable()
p2=PersonTable()

In [57]:
p1.first_name='Andrew'

In [58]:
p2.first_name

In [ ]:
from weakref import WeakKeyDictionary

Instance storage

In [59]:
class TextField:
    def __init__(self,length):
        self.length=length
        self._data={}
    def __get__(self,instance,owner):
        #return self._data.get(instance,None)
        return instance.__dict__.get('text_field_value',None)
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        #self.value=value
        #self._data[instance]=value 
        instance.__dict__['text_field_value']=value
    def __delete__(self,instance):
        pass
class PersonTable:
    first_name=TextField(200)
p1=PersonTable()
p2=PersonTable()

In [63]:
p1.first_name = "John"
p2.first_name='Bill'

In [64]:
p1.first_name

'John'

In [65]:
p2.first_name

'Bill'

In [66]:
p1.__dict__, p2.__dict__

({'text_field_value': 'John'}, {'text_field_value': 'Bill'})

In [67]:
class PersonTable:
    first_name=TextField(200)
    last_name=TextField(100)

In [68]:
p1=PersonTable()

In [69]:
p1.first_name = "John"
p1.last_name = "Smith"

In [70]:
p1.first_name # both the name and last name are same (Smith), cause we use same key for everything

'Smith'

In [71]:
p1.__dict__

{'text_field_value': 'Smith'}

In [89]:
class TextField:
    def __init__(self,length):
        self.length=length
        
    def __set_name__(self,owner,name): # not to specify name of an attr
        self.name=name
    def __get__(self,instance,owner):
        
        return instance.__dict__.get(f'TextField_{self.name}',None)
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        instance.__dict__[f"TextField_{self.name}"]=value
    def __delete__(self,instance):
        pass




In [90]:
class PersonTable:
    first_name=TextField(200)
    last_name=TextField(100)



In [91]:
p1=PersonTable()
p2=PersonTable()


In [92]:
p1.first_name='Andrew'
p1.last_name='Svetlov'
p1.first_name,p1.last_name

('Andrew', 'Svetlov')

Tying up loose ends

In [104]:
class TextField:
    def __init__(self,length):
        self.length=length
        
    def __set_name__(self,owner,name): # not to specify name of an attr
        print(owner)
        self.name=name
    def __get__(self,instance,owner):
        # print(instance)
        # print(owner)
        if instance is None:
            return self
        return instance.__dict__.get(f'TextField_{self.name}',None)
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        instance.__dict__[f"TextField_{self.name}"]=value
    def __delete__(self,instance):
        del instance.__dict__[f"TextField_{self.name}"]
# owner refers to the class itself
# instance reflectes intance from which descriptor it is used


In [105]:
class PersonTable:
    first_name=TextField(200)
    last_name=TextField(100)



<class '__main__.PersonTable'>
<class '__main__.PersonTable'>


In [106]:
PersonTable.first_name

In [107]:
p1=PersonTable()

In [108]:
p1.first_name="John"
p1.last_name="Doe"


In [109]:
del p1.first_name

In [110]:
p1.first_name

Non-data descriptors

In [121]:
class TextField:
    def __init__(self,length):
        self.length=length
        
    def __set_name__(self,owner,name): 
        
        self.name=name
    def __get__(self,instance,owner):
        if instance is None:
            return self
        return instance.__dict__.get(f'TextField_{self.name}',None)
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        instance.__dict__[f"TextField_{self.name}"]=value
    def __delete__(self,instance):
        del instance.__dict__[f"TextField_{self.name}"]
class PersonTable:
    first_name=TextField(200)
    last_name=TextField(100)

# non-data descriptor is the descriptor that defines one method - __get__



In [122]:
from random import randint
class LuckyNum:
    def __get__(self,instance,owner):
        return randint(1,100)

In [123]:
class PersonTable:
    first_name=TextField(200)
    last_name=TextField(100)
    personal_num=LuckyNum()
    def __init__(self,personal_num):
        self.personal_num=personal_num

In [125]:
p=PersonTable(10) # now it uses not getter, but a value we specified, because non-data descriptors come secondary

In [130]:
p.personal_num

10

Properties vs descriptors

In [ ]:
class TextField:
    def __init__(self,length):
        self.length=length
        
    def __set_name__(self,owner,name): 
        
        self.name=name
    def __get__(self,instance,owner):
        if instance is None:
            return self
        return instance.__dict__.get(f'TextField_{self.name}',None)
    def __set__(self,instance,value):
        if not type(value) == str:
            raise TypeError("value must be string")
        if len(value) > self.length:
            raise ValueError("value is too long")
        instance.__dict__[f"TextField_{self.name}"]=value
    def __delete__(self,instance):
        del instance.__dict__[f"TextField_{self.name}"]
class PersonTableWithDescriptors:
    first_name=TextField(200)





In [131]:
class PersonTableWithProps:
    def __init__(self,first_name_length):
        self._TextField_first_name=None
        self.first_name_length = first_name_length
    @property
    def first_name(self):
        return self._TextField_first_name
    @first_name.setter
    def first_name(self,value):
        if not isinstance(value,str):
            raise TypeError('first name must be a string')
        if len(value)>self.first_name_length:
            raise ValueError('first name is too long')
        self._TextField_first_name=value
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name


In [135]:
p=PersonTableWithProps(200)
p.first_name='John'

In [133]:
p.first_name

'John'

In [137]:
p.__dict__['first_name'] = "won't be able to reach"

In [138]:
p.__dict__

{'_TextField_first_name': 'John',
 'first_name_length': 200,
 'first_name': "won't be able to reach"}

In [139]:
p.first_name

'John'

In [ ]:
# if we want to add new attr, we have to repeat ourself 